In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, morphology
import gudhi as gd
import cv2 as cv

In [2]:
plt.get_backend()
# plt.rcParams.update({
    # "text.usetex": True,
    # "text.latex.engine": "pdflatex",
    # "text.latex.preamble": r"""
    # """,
# })

'module://matplotlib_inline.backend_inline'

In [ ]:

# ----------------------------
# 1. Загрузка или создание изображения
# ----------------------------

# Вариант A: загрузить своё изображение
# image = io.imread('plantogram.png', as_gray=True)

# Вариант B: создать искусственную плантограмму (для демонстрации)
def create_synthetic_plantogram():
    # Создаём чёрный фон
    img = np.zeros((200, 150))
    # Пятка — эллипс снизу
    rr, cc = skimage.draw.ellipse(160, 75, 30, 20, shape=img.shape)
    img[rr, cc] = 1
    # Плюсна — прямоугольник
    img[80:150, 60:90] = 1
    # Пальцы — 5 кругов
    for i, x in enumerate([50, 65, 75, 85, 100]):
        rr, cc = skimage.draw.disk((40, x), 8, shape=img.shape)
        img[rr, cc] = 1
    # Добавим "петлю" между большим и вторым пальцем
    rr, cc = skimage.draw.line(40, 65, 40, 50)
    img[rr, cc] = 1
    return img

# Используем синтетическое изображение
import skimage.draw
# plantogram = create_synthetic_plantogram()
plantogram = cv.imread("/home/dima/Documents/альбрехт/обработка изображений/default/images/Img (copy 7).png")
plantogram = cv.cvtColor(plantogram, cv.COLOR_RGB2GRAY)

plt.imshow(plantogram)

# Бинаризуем (если нужно): давление > 0 → область стопы
binary = plantogram > 0

# Инвертируем: GUDHI работает с "высокими" значениями как с присутствием
# Но для кубического комплекса мы передаём 1 где есть стопа, 0 — фон
# GUDHI ожидает, что 1 = "заполнено", 0 = "пусто"
image_for_gudhi = binary.astype(int)

# ----------------------------
# 2. Построение кубического комплекса
# ----------------------------
print("Строим кубический комплекс...")
cubical_complex = gd.CubicalComplex(
    dimensions=image_for_gudhi.shape,
    top_dimensional_cells=image_for_gudhi.flatten()
)

# ----------------------------
# 3. Вычисление персистентной гомологии
# ----------------------------
print("Вычисляем персистентную гомологию...")
# Фильтрация: пустые клетки (фон) имеют значение 1.0, заполненные — 0.0
# Но в нашем случае: мы хотим, чтобы стопа появлялась при пороге 0
# GUDHI по умолчанию считает, что ячейки с 1 появляются раньше
# Поэтому инвертируем значения:
#   фон = 1.0 (появляется поздно), стопа = 0.0 (появляется рано)
# Но проще: используем filtration как 1 - image
filtration_values = 1.0 - image_for_gudhi.astype(float)
cubical_complex = gd.CubicalComplex(
    dimensions=image_for_gudhi.shape,
    top_dimensional_cells=filtration_values.flatten()
)

# Вычисляем персистентность до размерности 1 (H0 и H1)
persistence = cubical_complex.persistence(homology_coeff_field=2, min_persistence=0)

# ----------------------------
# 4. Визуализация
# ----------------------------
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Исходное изображение
axes[0].imshow(binary, cmap='gray')
# axes[0].set_title('Бинарная плантограмма')

axes[0].set_title('Binary platntogramm')
axes[0].axis('off')

# Баркод
gd.plot_persistence_barcode(persistence)
# plt.title('Персистентный баркод')
plt.title('Persistancy barcode')

# Сохраняем график баркода отдельно, так как plot_persistence_barcode создаёт свой график
# Поэтому перестроим аккуратнее:

# Аккуратная визуализация
fig2, axes2 = plt.subplots(1, 2, figsize=(12, 4))

axes2[0].imshow(binary, cmap='gray')
# axes2[0].set_title('Плантограмма')
axes2[0].set_title('Plantogramm')
axes2[0].axis('off')

gd.plot_persistence_diagram(persistence, axes=axes2[1])
# axes2[1].set_title('Персистентная диаграмма')

axes2[1].set_title('Persistancy diagram')

# plt.tight_layout()
plt.show()

# ----------------------------
# 5. Анализ результатов
# ----------------------------
# Разделим H0 и H1
h0 = [p for p in persistence if p[0] == 0]  # компоненты связности
h1 = [p for p in persistence if p[0] == 1]  # петли

print(f"\nНайдено {len(h0)} компонент связности (H0).")
print(f"Найдено {len(h1)} топологических петель (H1).")

# Фильтрация по "персистентности" (разнице birth-death)
# Шум обычно имеет малую персистентность
min_pers = 2  # порог персистентности (в пикселях/условных единицах)

persistent_h0 = [p for p in h0 if p[1][1] - p[1][0] > min_pers or np.isinf(p[1][1])]
persistent_h1 = [p for p in h1 if p[1][1] - p[1][0] > min_pers or np.isinf(p[1][1])]

print(f"\nУстойчивых компонент (персистентность > {min_pers}): {len(persistent_h0)}")
print(f"Устойчивых петель (персистентность > {min_pers}): {len(persistent_h1)}")

# Пример интерпретации:
if len(persistent_h1) >= 1:
    print("\n→ Обнаружена устойчивая петля! Возможно, это область между пальцами.")
else:
    print("\n→ Устойчивых петель не обнаружено.")